In [2]:
#uncomment to install packages
#conda install -c conda-forge selenium
#!pip install fake_useragent
#!pip install eventlet
#!pip install python-settings

In [3]:
from bs4 import BeautifulSoup
import requests
import time, os
import re
import random
from random import randint
import random
from time import sleep

import pandas as pd
from pymongo import MongoClient
from pprint import pprint
import json

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [5]:
from fake_useragent import UserAgent

In [40]:
#helper files
import get_proxies as proxies
import extractors

### Create Mongo collection for urls

In [6]:
client = MongoClient()

In [7]:
db = client['philadelphia_home_prices']

In [8]:
db.list_collection_names()

['prices_messy', 'listing_urls', 'listing_data']

In [9]:
#uncomment to delete the collection
#db.drop_collection('listing_urls')

In [10]:
db.list_collection_names()

['prices_messy', 'listing_urls', 'listing_data']

In [11]:
col = db.create_collection('listing_urls')

CollectionInvalid: collection listing_urls already exists

In [12]:
db.list_collection_names()

['prices_messy', 'listing_urls', 'listing_data']

### scrape pages for listing urls

In [13]:
url = 'https://www.realtor.com/'

In [14]:
# 44 listings per page. Choose start and stop page
page_from = 26
page_to = 26

In [15]:
col = db.get_collection('listing_urls')

In [16]:
col.count()

/Users/chrismalasics/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


1100

In [ ]:
for i in range(page_from, page_to+1):
    #print(i) #prints page that is open in case bot is detected
    driver = webdriver.Chrome(chromedriver)
    driver.get(url + 'soldhomeprices/Philadelphia_PA/pg-' + str(i))
    
    sleep(random.uniform(60,120))
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    page = str(soup.find_all('div', class_='photo-wrap'))
    extensions = re.findall(r'[A-z]+-[A-z]+\/[0-9-A-z]+[^"]', page)
    
    if len(extensions) == 0:
        print('bot is caught on page {}'.format(i))
        break
        
    for i in extensions:
        page_url = url + i
        page = {'url': page_url}
        col.insert_one(page)
        
    sleep(random.uniform(1,3))
    
    driver.close()

In [17]:
col.count()

/Users/chrismalasics/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


1100

### Pull urls from mongoDB into memory

In [18]:
#pull urls from mongo db
cursor = db.listing_urls.find({})
listings = list(cursor)

In [19]:
#convert dictionary values to list
listing_urls_list = []
for i in listings:
    url = i['url']
    listing_urls_list.append(url)

In [20]:
#see first 5 in list
listing_urls_list[0:5]

['https://www.realtor.com/realestateandhomes-detail/1232-N-Mascher-St-Apt-A_Philadelphia_PA_19122_M49182-95881',
 'https://www.realtor.com/realestateandhomes-detail/212-Carpenter-St-Rear-C_Philadelphia_PA_19147_M34206-29412',
 'https://www.realtor.com/realestateandhomes-detail/1211-E-Cheltenham-Ave_Philadelphia_PA_19124_M33700-20336',
 'https://www.realtor.com/realestateandhomes-detail/1002-Brandywine-St_Philadelphia_PA_19123_M39147-93479',
 'https://www.realtor.com/realestateandhomes-detail/2607-N-31st-St_Philadelphia_PA_19132_M32883-74202']

### Get data from urls

#### new collection for listing data

In [21]:
#uncomment the code you need to use
#db.create_collection('listing_data')
col = db.get_collection('listing_data')

In [22]:
db.collection_names()

/Users/chrismalasics/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['prices_messy', 'listing_urls', 'listing_data']

In [23]:
#uncomment to drop
#db.drop_collection('listing_data')

In [24]:
db.collection_names()

/Users/chrismalasics/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['prices_messy', 'listing_urls', 'listing_data']

#### Rotate proxies

In [38]:
#get list of proxies from free-proxy.net
proxies_address = proxies.proxy_list()

2020-11-03 15:23:49,574 root   DEBUG    === Initialized Proxy Parsers ===
2020-11-03 15:23:49,575 root   DEBUG    	 FreeProxy parser of 'http://free-proxy-list.net' with required bandwidth: '150' KBs
2020-11-03 15:23:49,575 root   DEBUG    	 PremProxy parser of 'https://premproxy.com/list/' with required bandwidth: '150' KBs
2020-11-03 15:23:49,576 root   DEBUG    =================================
2020-11-03 15:23:50,048 root   DEBUG    Added 300 proxies from FreeProxy
2020-11-03 15:23:50,621 http_request_randomizer.requests.parsers.PremProxyParser DEBUG    Pages: {'', '06.htm', '03.htm', '08.htm', '05.htm', '10.htm', '04.htm', '09.htm', '07.htm', '02.htm'}
2020-11-03 15:23:51,202 http_request_randomizer.requests.parsers.js.UnPacker INFO     JS UnPacker init path: https://premproxy.com/js/394a0.js
2020-11-03 15:23:51,787 http_request_randomizer.requests.parsers.js.UnPacker DEBUG    portmap: {'re356': '8080', 'r1000': '32108', 'ra8f9': '51671', 'rdf03': '8087', 're2a2': '8000', 'r7241':

In [27]:
listing_from = 12
listing_to = len(listing_urls_list)
count = 0
good_proxies = []
bad_proxies = []
bad_url = []

In [39]:
import sys
import concurrent.futures
from datetime import datetime
import eventlet
import settings

In [ ]:
pool = eventlet.GreenPool(settings.max_threads)
pile = eventlet.GreenPile(pool)

In [ ]:
datetime.now()

In [ ]:
MAX_THREADS = 30

for i in range(listing_from, listing_to+1): 
    print(i, listing_urls_list[i])

    try:
        PROXY = proxies[count].get_address()
        webdriver.DesiredCapabilities.CHROME['proxy']={
                                                        "httpProxy":PROXY,
                                                       "ftpProxy":PROXY,
                                                       "sslProxy":PROXY,
                                                       "proxyType":"MANUAL",
                                                        }
        driver = webdriver.Chrome(chromedriver)
        driver.get(listing_urls_list[i])

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        listing_data = get_data(soup)
        good_proxies.append(proxy)

    except:
        bad_proxies.append(proxy)
        count += 1
        driver.close()
        
    driver.close()
    
    #insert listing data into mongo database
    col.insert_one(listing_data)
    print(listing_data)
    count += 1
    print(count)

In [ ]:
threads = min(MAX_THREADS, len(story_urls))

with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    executor.map(download_url, story_urls)

In [ ]:
count

In [29]:
for i in range(listing_from, listing_to+1): 
    print(i, listing_urls_list[i])
    check = False
    state = False
    while state==False:
        try:
            PROXY = proxies[count].get_address()
            print(PROXY)
            webdriver.DesiredCapabilities.CHROME['proxy']={
                                                            "httpProxy":PROXY,
                                                           "ftpProxy":PROXY,
                                                           "sslProxy":PROXY,
                                                           "proxyType":"MANUAL",
                                                            }
            driver = webdriver.Chrome(chromedriver)
            driver.get(listing_urls_list[i])

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            listing_data = get_data(soup)
            
            try:
                check = (soup.find('span', itemprop="streetAddress")).text
            except:
                bad_url.append(listing_urls_list[i])
                break
      
                
            state = True
            tested_proxies.append(proxy)
        except:
            bad_proxies.append(proxy)
            count += 1
            driver.close()
        
    driver.close()
    
    #insert listing data into mongo database
    col.insert_one(listing_data)
    print(listing_data)
    count += 1
    print(count)
    
    
    
    

    
    #if i//5 == 0:
    #    sleep(random.uniform(60,120))
        
    ####     
    #session=requests.Session()
    #session.headers = user_agent#{'User-Agent': ua.random} #'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    #print(user_agent)
    ##base_url='https://www.realtor.com'
    ##re_list=[]
    ##realtor_default_count=44
    ##page_counter=1
    ##search_url=''.join([base_url, '/realestateandhomes-search/Sacramento_CA'])
    #page=session.get(listing_urls_list[i])
    #c=page.content
    #soup=BeautifulSoup(c,"html.parser")
    ####
    
    #options = Options()
    #ua = UserAgent()
    #userAgent = ua.random
    #print(userAgent)
    #options.add_argument(f'user-agent={userAgent}')
    #driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
    #driver.get(listing_urls_list[i])
    
    #sleep(random.uniform(120,180)) #pause

12 https://www.realtor.com/realestateandhomes-detail/731-Thomas-Rd_Philadelphia_PA_19118_M35952-99959


NameError: name 'proxy' is not defined

In [ ]:
hello = {'hi':[]}

In [ ]:
hello['hi'] == []

In [ ]:
for i in range(listing_from, listing_to+1): 
    
    print(i, listing_urls_list[i])
    
    #ua = UserAgent()
    #user_agent = {'User-agent': ua.random}
    #print(user_agent)

    #response  = requests.get(listing_urls_list[i], headers = user_agent)
    
    #driver = webdriver.Chrome(chromedriver)
    #driver.get(listing_urls_list[i])
    
    #if i//5 == 0:
    #    sleep(random.uniform(60,120))
        
    ####     
    #session=requests.Session()
    #session.headers = user_agent#{'User-Agent': ua.random} #'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    #print(user_agent)
    ##base_url='https://www.realtor.com'
    ##re_list=[]
    ##realtor_default_count=44
    ##page_counter=1
    ##search_url=''.join([base_url, '/realestateandhomes-search/Sacramento_CA'])
    #page=session.get(listing_urls_list[i])
    #c=page.content
    #soup=BeautifulSoup(c,"html.parser")
    ####
    
    #options = Options()
    #ua = UserAgent()
    #userAgent = ua.random
    #print(userAgent)
    #options.add_argument(f'user-agent={userAgent}')
    #driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
    #driver.get(listing_urls_list[i])

    ###
    
    '''options = Options()
    ua = UserAgent()
    userAgent = ua.random
    #print(userAgent)
    options.add_argument(f'user-agent={userAgent}')
    driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
    driver.get(listing_urls_list[i])
    #user_agent = {'User-agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}
    #response  = requests.get(listing_urls_list[i], headers = user_agent)
    
    sleep(random.uniform(60,100))'''
    ###
    
   
    
    PROXY = proxies[count].get_address()
    
    
    print(PROXY)
    webdriver.DesiredCapabilities.CHROME['proxy']={
                                                    "httpProxy":PROXY,
                                                    "ftpProxy":PROXY,
                                                    "sslProxy":PROXY,

                                                    "proxyType":"MANUAL",

                                                    }
    driver = webdriver.Chrome(chromedriver)
    driver.get(listing_urls_list[i])
    
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    

    driver.close()
    
    #insert listing data into mongo database
    col.insert_one(listing_data)
    print(listing_data)
    count += 1
    
    #sleep(random.uniform(120,180)) #pause

In [ ]:
proxies

In [ ]:
listing_data

In [ ]:
test = ('hi',50)

In [ ]:
test[1]

In [ ]:
type(test)

In [ ]:
def test(num):
    return num+12, num-12

In [ ]:
plus, minus = test(12)

In [ ]:
plus

In [ ]:
minus

In [ ]:
proxies[1].get_address()

In [ ]:
proxies

In [ ]:
PROXY = proxies[0].get_address()

In [ ]:
PROXY

In [ ]:
webdriver.DesiredCapabilities.CHROME['proxy']={
                                                            "httpProxy":PROXY,
                                                           "ftpProxy":PROXY,
                                                           "sslProxy":PROXY,
                                                           "proxyType":"MANUAL",
                                                            }

In [ ]:
driver = webdriver.Chrome(chromedriver)
driver.get(listing_urls_list[0])

In [ ]:
listing_urls_list[0]

In [ ]:
type(listing_urls_list)

In [ ]:
listing_urls_list[:20]

In [ ]:
soup

In [ ]:
try:
    check = (soup.find('span', itemprop="streetAddress")).text
except:

In [ ]:
proxies_address